In [1]:
from pandas_ext.UpdatedDataFrame2 import UpdatedDataFrame as DataFrame
import pandas as pd
from IPython.core.display import display, HTML

In [2]:
%%html
<style>
.katex { font-size: 40px; }
</style>

In [3]:
script = \
"""
<script>
requirejs.config({
    paths: { 
       'katex': ['//cdnjs.cloudflare.com/ajax/libs/KaTeX/0.9.0-beta1/katex.min'], 
                                              // strip .js ^, require adds it back
    },
});

require(['katex'], function(katex) {
     console.log("Loaded :)");
     console.log(katex)
     window.katex = katex
     return {};
});
</script>
"""

In [4]:
display(HTML(script))

In [5]:
import requests
r = requests.get('https://cdnjs.cloudflare.com/ajax/libs/KaTeX/0.9.0-beta1/katex.min.css')

In [6]:
sty = r.content.decode('utf8')
# sty = sty.replace('fonts/', 'https://cdnjs.cloudflare.com/ajax/libs/KaTeX/0.9.0-beta1/fonts/')
display(HTML("<style>"+sty+"</style>"))

In [42]:
from IPython.display import (display_javascript,
                                         display_html)
from uuid import uuid1
import json

class katex(object):
    def __init__(self, s):
        self._s = json.dumps(s)
        self.dep = "katex"
        self._ids = []
        self._katex = ("var katex_try = function(s, el) {\n"
                       "\t try {"
                       "\t\twindow.katex.render(s, el);\n"
                       "} catch(err) {\n"
                       "\t\t el.innerHTML = s;\n"
                       "\t}\n}")
        self._ctx = lambda body: "\nvar ctx = function() {\n"+body+"\n}\nctx();\n"
        self._whenavailable_func = ("var whenAvailable = function(name, callback) {\n"
                                "\tvar interval = 10; // ms\n"
                                "\twindow.setTimeout(function() {\n"
                                "\tif (window[name]) {\n"
                                "\t\tcallback();\n"
                                "\t} else {\n"
                                "\t\twindow.setTimeout(arguments.callee, interval);\n"
                                "\t}\n"
                                "}, interval);\n"
                                "}\n")
        _parent = 'var parent = document.getElementById(id);'
        _body = lambda math: "\n".join(["var newChild = document.createElement('span');",
                                        'katex_try({math}, newChild);'.format(math=math),
                                        "parent.appendChild(newChild);"])
        self._fn = lambda math: "var render_katex = function(id) { \n\t"+_parent+"\n\t"+_body(math)+"\n}"
        self._exec = lambda uid: 'whenAvailable("katex", function(){render_katex("'+uid+'");});'
#         self._exec = lambda uid: 'render_katex("'+uid+'");'
        self._filled_ctx = lambda uid, math: self._ctx("\n".join([self._whenavailable_func, self._katex,
                                                                  self._fn(math), self._exec(uid)]))
        
    def _repr_html_(self):
        unique_id = str(uuid1())
        anchor = '<div id="{unique_id}"/>'.format(unique_id=unique_id)
        self._ids.append(unique_id)
        return anchor

    def _repr_javascript_(self):
        js = []
        while len(self._ids) > 0:
            i = self._ids.pop(0)
            funcs = self._filled_ctx(i, self._s)
            js.append(funcs)
        return "\n".join(js)
    
    def _ipython_display_(self):
        display_html(self._repr_html_(), raw=True)
        display_javascript(self._repr_javascript_(), raw=True)

In [44]:
k = katex(r"\int x")
k

k2 = katex(r"e = mc^2 \frac{\lambda y + 100}{\int e = mc^2} = \sqrt{600}")
k

In [45]:
k2

In [46]:
df_5 = DataFrame([k, k2])
df_1 = DataFrame([df_5, df_5])
DataFrame([df_1, df_1])

0 
 
 
 
 
 0 
 

 
 
 
 
 0 
 
 
 
 
 0 
 

 
 
 
 
 0 
 
 
 
 
 0 
 
 
 
 1 
 
 
 
 
 
 
 
 1 
 

 
 
 
 
 0 
 
 
 
 
 0 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 1 
 

 
 
 
 
 0 
 
 
 
 
 0 
 

 
 
 
 
 0 
 
 
 
 
 0 
 
 
 
 1 
 
 
 
 
 
 
 
 1 
 

 
 
 
 
 0 
 
 
 
 
 0 
 
 
 
 1

In [47]:
df = DataFrame(pd.read_csv("eqs_100k.tsv", sep = "\t", header=None))
df.columns = ["eq_id", "eq"]

In [52]:
clean = df['eq'].str.replace(r'\\\\begin{equation}', "").str.replace(r'\\\\end{equation}', "")
clean = clean.str.replace(r'\\n', "").str.replace(r'\\t', "")
clean = clean.str.replace(r"\\\\", r"\\")
df['clean'] = clean

In [68]:
df['katex'] = df['clean'].apply(lambda x: katex(x))
# so we can get a better idea of what we're looking at - omits obnoxiously long strings
df[df['eq'].str.len() < 400][['katex']].head(40)

,katex
0,
1,
2,
3,
4,
5,
6,
7,
8,
9,
